In [113]:
import pandas as pd

stores_train = pd.read_csv('data/stores_train.csv')
stores_test = pd.read_csv('data/stores_test.csv')
plaace_hierarchy = pd.read_csv('data/plaace_hierarchy.csv')


stores_train_copy= stores_train.copy()
stores_test_copy= stores_test.copy()

plaace_hierarchy_copy = plaace_hierarchy.copy()
plaace_hierarchy_copy.drop(columns=['sales_channel_name'], inplace=True)

plaace_hierarchy_copy['lv2'] = plaace_hierarchy_copy['lv2'].astype('category')
plaace_hierarchy_copy['lv2'] = plaace_hierarchy_copy['lv2'].astype('category')

df = stores_train_copy

df = pd.merge(df, plaace_hierarchy_copy, how='left', on='plaace_hierarchy_id')

busstops = pd.read_csv('data/busstops_norway.csv')
busstops_copy = busstops.copy()
busstops_copy['busstop_lat'] = busstops_copy['geometry'].apply(lambda x: float(x.split(' ')[1][:-1]))
busstops_copy['busstop_lon'] = busstops_copy['geometry'].apply(lambda x: float(x.split(' ')[0][6:]))

In [1]:
from haversine import haversine, Unit
import time
import numpy as np


def find_closest_busstop(lat, lon):
    distances = busstops_copy.apply(lambda x: haversine((lat, lon), (x['busstop_lat'], x['busstop_lon']), unit=Unit.METERS), axis=1)
    return distances.min()


stores_train_copy['closest_busstop'] = stores_train_copy.apply(lambda x: find_closest_busstop(x['lat'], x['lon']), axis=1)
stores_test_copy['closest_busstop'] = stores_test_copy.apply(lambda x: find_closest_busstop(x['lat'], x['lon']), axis=1)


stores_train_copy.to_csv('data/stores_train_with_busstops.csv', index=False)
stores_test_copy.to_csv('data/stores_test_with_busstops.csv', index=False)


In [ ]:
from haversine import haversine, Unit

def count_closest_stores(lat, lon, lv2, col, radius):
    distances = df[df[col] == lv2].apply(lambda x: haversine((lat, lon), (x['lat'], x['lon']), unit=Unit.METERS), axis=1)
    return len(distances[distances < radius] > 0)

radius = 500

col = 'lv2_desc'

df_train['num_closest_stores_lv2'] = df_train.apply(lambda x: count_closest_stores(x['lat'], x['lon'], x['lv2_desc'], col, 500), axis=1)
df_test['num_closest_stores_lv2'] = df_test.apply(lambda x: count_closest_stores(x['lat'], x['lon'], x['lv2_desc'], col, 500), axis=1)

df_train[['store_id','num_closest_stores_lv2_500m']].to_csv('stores_train_num_closest_stores_lv2_500m.csv', index=False)
df_test[['store_id','num_closest_stores_lv2_500m']].to_csv('stores_test_num_closest_stores_lv2_500m.csv', index=False)
